# Test de modèles


@Author Colin de Seroux


## <span style="color:lightblue">Installation des dépendances</span>


In [ ]:
%pip install -q -r requirements.txt

## <span style="color:lightblue">Importation des dépendances</span>


In [2]:
import json
from ultralytics import YOLO
import torch

## <span style="color:lightblue">Code principale</span>


### <span style="color:lightgreen">Environnement</span>


In [9]:
DATASET_PATH = "../data/dataset4"
MODEL_NAME = "yolov8n_trained.pt"
PLATFORM = "i7-11800H"

### <span style="color:lightgreen">Chargement des images</span>


In [10]:
with open(f"{DATASET_PATH}/test/result.json", 'r') as f:
    coco_data = json.load(f)

images = coco_data["images"]

### <span style="color:lightgreen">Chargement du modèle</sapn>


In [ ]:
model = YOLO(f"./saved/{MODEL_NAME}")

### <span style="color:lightgreen">Prédictions</span>


Chargement du type d'hardware.


In [ ]:
device = 0 if torch.cuda.is_available() else "cpu" # ou tpu:0 ou dla:0

print(f"Device: {device}")

Lancement des prédictions.


In [ ]:
predictions = []

for image in images:
    results = model.predict(f"{DATASET_PATH}/test/{image['file_name']}", device=device)
    
    inference_time = results[0].speed["inference"]
    
    for result in results:
        if result.boxes:
            for box in result.boxes:
                cls = box.cls.item()
                conf = box.conf.item()
                boxes = box.xyxy.tolist()[0]
                predictions.append({
                    "image_id": image["id"],
                    "category_id": int(cls),
                    "bbox": [boxes[0], boxes[1], boxes[2] - boxes[0], boxes[3] - boxes[1]],
                    "score": conf,
                    "inference_time": inference_time
                })

Sauvegarde des prédictions dans un fichier.


In [14]:
with open(f"../results/predictions/predictions_{PLATFORM}_{MODEL_NAME}.json", 'w') as f:
    json.dump(predictions, f)

## <span style="color:lightblue">Sources</sapn>
